In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE orders_raw
COMMENT "The raw book orders, ingested from orders_raw "
AS SELECT * FROM cloud_files(
  "dbfs:/mnt/demo-datasets/bookstore/orders-raw", 
  "parquet", 
  map("schema", "order_id STRING, order_timestamp LONG, customer_id STRING, quantity LONG"))

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE customers
COMMENT "The customers lookup table, ingested from customers_json "
AS SELECT * FROM json.`dbfs:/mnt/demo-datasets/bookstore/customers-json`

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE orders_cleaned(
CONSTRAINT valid_order_number EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW
) 
COMMENT "The cleaned books orders with valid order_id"
AS SELECT order_id, quantity, o.customer_id, c.profile:first_name AS f_name, c.profile:last_name AS l_name, CAST(TO_TIMESTAMP(order_timestamp) AS TIMESTAMP) order_timestamp, c.profile:address:country AS country FROM STREAM(LIVE.orders_raw) o
LEFT OUTER JOIN LIVE.customers c ON o.customer_id = c.customer_id

In [0]:
%sql
CREATE OR REFRESH LIVE TABLE cn_daily_customer_books
COMMENT "Daily number of books per customer in China"
AS 
  SELECT customer_id, f_name, l_name, date_trunc("DD", order_timestamp) order_date, 
  sum(quantity) books_counts 
  FROM LIVE.orders_cleaned
  WHERE country = "China"
  GROUP BY customer_id, f_name, l_name, date_trunc("DD", order_timestamp)